In [73]:
import numpy as np
import torch
import torch.nn as nn
import time

In [91]:
class FNN(nn.Module):
    def __init__(self,layers,activation,X_max=None,X_min=None,in_tf=None,out_tf=None,BC=None):
        super().__init__()
        self.activation = activation
        self.linears = nn.ModuleList()
        for i in range(1,len(layers)):
            self.linears.append(nn.Linear(layers[i-1],layers[i]))
            nn.init.xavier_uniform_(self.linears[-1].weight)
            nn.init.zeros_(self.linears[-1].bias)
        self.last = nn.Softplus()
        
        self.X_max = X_max
        self.X_min = X_min
        self.in_tf = in_tf
        self.out_tf = out_tf
        self.BC = BC
                      
    def forward(self,inputs):
        X = inputs
        # input transformation
        if self.in_tf:
            X = self.in_tf(X,self.X_max,self.X_min)
        # linear layers    
        for linear in self.linears[:-1]:
            X = self.activation(linear(X))
        # last layer
        X = self.last(self.linears[-1](X))
        # output transformation
        if self.out_tf:
            X = self.out_tf(X,self.BC,inputs,self.X_max,self.X_min)
        return X

In [92]:
def input_transform(X,X_max,X_min):
    X = 2.*(X-X_min)/(X_max-X_min) - 1.
    return X

def output_transform(X,hard_BC=None,inputs=None,X_max=None,X_min=None):
    X = 3000*X
    if hard_BC:
        X = X*hard_BC(inputs,X_max,X_min)
    return X

In [93]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net =  FNN([4,64,64,64,1],nn.Tanh(),X_max=torch.tensor([45.,3.,30.,30]),X_min=torch.tensor([0.,0.,0.,0.]),
                      in_tf=input_transform,out_tf=output_transform)
params = net.parameters()
optimizer = torch.optim.Adam(params,lr=4e-4)

In [94]:
T = np.load('./data/processed_data.npy')[13754:13754+1500:5,:,:]
x = np.linspace(0.375,44.625,60)
z = np.linspace(29.625,0.375,40)
t = np.linspace(13754/50.124635,13754/50.124635+1500/50.0124635,300)
[x,t,z] = np.meshgrid(x,t,z)
y = np.zeros_like(x)

data_output = torch.tensor(T.reshape(-1,1),requires_grad=True,dtype=torch.float).to(device)
data_input = torch.tensor(np.vstack((x.reshape(-1),y.reshape(-1),z.reshape(-1),t.reshape(-1)))
                          ,requires_grad=True,dtype=torch.float).to(device).T

In [95]:
iterations = 10000
l = np.zeros(iterations)
start_time = time.time()
for epoch in range(iterations):
    optimizer.zero_grad()
    T_pred = net(data_input)
    loss = nn.MSELoss()(T_pred,data_output)/9e6
   
    loss.backward() 
    optimizer.step() 
    
    if epoch%20 == 0:
        elapsed = time.time() - start_time
        print('It: %d, Loss: %.3e, Time: %.3e' % (epoch, loss, elapsed))
        start_time = time.time()

It: 0, Loss: 6.305e-02, Time: 1.894e+00
It: 20, Loss: 9.149e-03, Time: 3.479e+01
It: 40, Loss: 9.126e-03, Time: 3.486e+01
It: 60, Loss: 8.127e-03, Time: 3.488e+01
It: 80, Loss: 7.450e-03, Time: 3.484e+01


KeyboardInterrupt: 